# Pokerole SRD Builder

You don't need to use this, if you're looking for the Obsidian SRD, it's [here](https://github.com/Willowlark/PokeroleObsidianSRD).

In [1]:
import yaml
from glob import glob
import json
import os
import pandas as pd
import re
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from shutil import copy

secrets = json.load(open('../secrets.json'))

pokedex_path = '../Version20/Pokedex'
abilities_path = '../Version20/Abilities'
moves_path = '../Version20/Moves'
learnsets_path = '../Version20/Learnsets'
natures_path = '../Version20/Natures'

pokedex_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Pokedex/'
abilities_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Abilities/'
moves_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Moves/'
learnsets_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Learnsets/'
natures_output = secrets["ObsidianRoot"]+'/Pokerole SRD/SRD-Natures/'

os.makedirs(pokedex_output,exist_ok=True)
os.makedirs(abilities_output,exist_ok=True)
os.makedirs(moves_output,exist_ok=True)
os.makedirs(learnsets_output,exist_ok=True)
os.makedirs(natures_output,exist_ok=True)

/Users/bill/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
ability_template = '''## `= this.name`

> *`= this.Description`*

**Effect:** `= this.Effect`'''

#-------------------------------------------------------------

moves_template = '''### `= this.name` 
*`= this.Description`*

**Accuracy:** `= this.Accuracy1` + `= this.Accuracy2`
**Damage:** `= this.Power` `= choice(length(this.Damage1)=0, "","\+ "+ this.Damage1)` `= choice(length(this.Damage2)=0, "","\+ "+ this.Damage2)`

| Type          | Target          | Damage Type          | Power          |
| ------------- | --------------- | ---------------- | -------------- |
| `= this.Type` | `= this.Target` | `= this.DmgType` | `= this.Power` | 

**Effect:** `= this.Effect`'''

#-------------------------------------------------------------

learnsets_template = '''## `= this.Name` Learnset

**DexID:** `= this.DexID`

```dataview
TABLE WITHOUT ID
    T[0] AS Learned,
    T[1] AS Move
FROM #PokeroleSRD/Learnsets
flatten moves as T
where file.path = this.file.path
```
'''

#-------------------------------------------------------------------

natures_template = '''## `= this.Nature`

**Confidence**: `= this.Confidence`

*`= this.Keywords`*

> `= this.Description`
'''

# Pokedex Entries

In [15]:
for src in glob(pokedex_path+"/*.json"):
    entry = json.loads(open(src).read())
    entry['BoxSprite'] = 'SRD-BoxSprite-'+entry['Sprite']
    entry['HomeSprite'] = 'SRD-HomeSprite-'+entry['Sprite']
    
    entry['Unevolved'] = 'Yes' if entry['Unevolved'] else 'No'
    entry['HasForm'] = 'Yes' if entry['HasForm'] else 'No'
    entry['Legendary'] = 'Yes' if entry['Legendary'] else 'No'
    entry['GoodStarter'] = 'Yes' if entry['GoodStarter'] else 'No'
    entry['Baby'] = 'Yes' if entry['Baby'] else 'No'
    entry['Learnset'] = f"[[SRD-{entry['Name']}-Learnset]]"
    
    abilities = (f"[[SRD-{entry['Ability1']}|{entry['Ability1']}]]"
                        f"{'' if not entry['Ability2'] else ' / [[SRD-'+ entry['Ability2']+'|'+entry['Ability2']+']]'}"
                        f"{'' if not entry['HiddenAbility'] else ' ([[SRD-'+entry['HiddenAbility']+'|'+entry['HiddenAbility']+']])'}"
                        f"{'' if not entry['EventAbilities'] else ' <[[SRD-'+entry['EventAbilities']+'|'+entry['EventAbilities']+']]>'}"
                        )
    INTEGERS = ['BaseHP', 'Strength', 'MaxStrength',
       'Dexterity', 'MaxDexterity', 'Vitality', 'MaxVitality', 'Special',
       'MaxSpecial', 'Insight', 'MaxInsight']
    for key in INTEGERS:
        entry[key] = int(entry[key])

    template = f"""# `= this.name`

![[{entry['HomeSprite']}|right]]

*{entry['DexCategory']}*
*{entry['DexDescription']}*

**DexID**:: {entry['DexID']}
**Species**:: {entry['Name']}
**Type**:: {entry['Type']}
**Abilities**:: {abilities}
**Base HP**:: {entry['BaseHP']}

![[{entry['BoxSprite']}|right]]

|           |                                                                                        |                                          |
| --------- | -------------------------------------------------------------------------------------- | ---------------------------------------- |
| Strength  | `= padleft(padright("",this.MaxStrength - this.Strength,"⭘"),this.MaxStrength,"⬤")`    | (Strength::{entry['Strength']})/(MaxStrength::{entry['MaxStrength']})   |
| Dexterity | `= padleft(padright("",this.MaxDexterity - this.Dexterity,"⭘"),this.MaxDexterity,"⬤")` | (Dexterity:: {entry['Dexterity']})/(MaxDexterity::{entry['MaxDexterity']}) |
| Vitality  | `= padleft(padright("",this.MaxVitality - this.Vitality,"⭘"),this.MaxVitality,"⬤")`    | (Vitality::{entry['Vitality']})/(MaxVitality::{entry['MaxVitality']})   |
| Special   | `= padleft(padright("",this.MaxSpecial - this.Special,"⭘"),this.MaxSpecial,"⬤")`       | (Special::{entry['Special']})/(MaxSpecial::{entry['MaxSpecial']})     |
| Insight   | `= padleft(padright("",this.MaxInsight - this.Insight,"⭘"),this.MaxInsight,"⬤")`       | (Insight::{entry['Insight']})/(MaxInsight::{entry['MaxInsight']})     |

**Recommended Rank**:: {entry['RecommendedRank']}
**Good Starter**:: {entry['GoodStarter']}
**Can Evolve**:: {entry['Unevolved']}
**Height**: {str(entry['Height']['Feet']).split('.')[0]}'{str(entry['Height']['Feet']).split('.')[1]}" / {entry['Height']['Meters']}m
**Weight**: {entry['Weight']['Pounds']}lbs / {entry['Weight']['Kilograms']}kg

![[SRD-{entry['Name']}-Learnset]]"""
        
    for x in ['DexID','Strength','MaxStrength','Dexterity','MaxDexterity',
              'Vitality','MaxVitality','Special','MaxSpecial','Insight','MaxInsight',
             'BaseHP', 'RecommendedRank', 'GoodStarter', 'Unevolved', '_id', "Abilities", "Type"]:
        del entry[x]
    
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Pokedex\n\n{template}"
    open(pokedex_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)

# Ability Entries

In [22]:
for src in glob(abilities_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Abilities\n\n{ability_template}"
    open(abilities_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)

# Move Dex



In [23]:
for src in glob(moves_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Moves\n\n{moves_template}"
    open(moves_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)

# Move Lists

In [9]:
for src in glob(learnsets_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    
    entry['Species'] = f"[[SRD-{entry['Name']}|{entry['Name']}]]"
    moves = []
    for m in entry["Moves"]:
        if moves and m["Learned"] != moves[-1][0]:
            moves.append(["---------------------------","---------------------------"])
        moves.append([m[f'Learned'],f'[[SRD-{m["Name"]}|{m["Name"]}]]'])
    entry['Moves'] = moves
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Learnsets\n\n{learnsets_template}"
    open(learnsets_output+f"SRD-{entry['Name']}-Learnset.md",'w').write(entry_output)
    

# Natures

In [25]:
for src in glob(natures_path+"/*.json"):
    entry = json.loads(open(src).read())
    del entry['_id']
    entry_output = f"---\n{yaml.dump(entry)}---\n\n#PokeroleSRD/Natures\n\n{natures_template}"
    open(natures_output+f"SRD-{entry['Name']}.md",'w').write(entry_output)

# Sprites (No longer Automated)

In [26]:
# sprites = os.listdir('../../special-pancake/In/pokesprite/pokemon-gen8/regular')
# data = pd.read_csv(pokedex_path)
# BoxSprites = []
# for n in data.Name:
#     best_match = process.extractOne(n, sprites, scorer=fuzz.token_sort_ratio)
#     # print(n, best_match)
#     BoxSprites.append(best_match[0])
# data['BoxSprite'] = BoxSprites

In [27]:
# # x_x_DexNum_Form_Femaled/Maled/MaleFemale_Gigantamax_x_front/back_Normal/Rare
# # 0_1_2______3____4________________________5__________6_7__________8
# home = sorted(os.listdir('../../special-pancake/In/Pokémon HOME/'))
# data = pd.read_csv(pokedex_path)

# pattern = '^[a-z]+_[a-z]+_(\d+)_(\d+)_([a-z]+)_([a-z]+)_\d+_f_([a-z]+)'
# # Group 1 = Number
# # Group 2 = Form  
# # Group 3 = Gender (mf, md, fd, uk)
# # Group 4 = Gigantamax? (n/g)
# # Group 5 = Rarity (n/r)

# shiny = []
# gigantamax = []
# regular = []

# for f in home:
#     m = re.match(pattern,f)
#     if not m:
#         continue
#     if m.group(3) == 'fd':
#         continue
#     if m.group(4) == 'g':
#         gigantamax.append(f)
#         continue
#     if m.group(5) == 'r':
#         shiny.append(f)
#         continue
#     regular.append(f)
# print("Shiny:", len(shiny), "Giga:", len(gigantamax), "Regular:", len(regular))

# sprites = pd.Series(regular).to_frame()
# sprites['num'] = sprites[0].str.extract('^[a-z]+_[a-z]+_(\d+)_\d+')
# sprites['form'] = sprites[0].str.extract('^[a-z]+_[a-z]+_\d+_(\d+)')
# sprites['key'] = sprites['num'].astype('str')+sprites['form'].astype('int').astype('str')


# data['num'] = data['DexID'].str.extract('^(\d+)')
# data['form'] = data['DexID'].str.extract('^\d+[A-Z]+(\d+)')
# data.loc[data.DexID.str.isdigit(), 'form'] = '0'
# data.loc[data.DexID.str.match('\d+[A-Z]+$'), 'form'] = '1'

# # sprites.to_csv('sprites.csv',index=0)
# # data.to_csv('output.csv',index=0)

# data['key'] = data['num'].astype('str')+data['form'].astype('int').astype('str')

# data = data.merge(sprites, on='key')
# data.apply(lambda x: shutil.copy('../../special-pancake/In/Pokémon HOME/'+x[0], '../../special-pancake/In/Pokémon HOME new/SRD-HomeSprite-'+x.Sprite),axis=1)
# data.to_csv('output.csv',index=0)

In [28]:
# # SRD-Home goes to HomeSprites
# # SRD-Sprites goes to BoxSprites

# for src in glob(secrets["ObsidianRoot"]+"/Pokerole SRD/SRD-Home/*.png"):
#     #print(src)
#     x = re.search("SRD-\w+Sprite-(.*)", src).group(1)
#     copy(src, '/Users/bill/Code/Pokerole Data/data/BoxSprites/'+x)